### Note: Using this notebook
This notebook walks through the available data, and introduces you to the concepts and tools that you can use in order to prepare, propose, and solve data science problems. Each code cell in this tool can be executed to replicate the results. 

For tips/tricks on using Jupyter Notebooks, please see: https://jupyter-notebook.readthedocs.io/en/stable/examples/Notebook/Notebook%20Basics.html

# Data Exploration
## Load Packages

First, let's take a look at the data we have downloaded, in order to do this, we will import a few pacakges

Pandas: To Play with data frames (kind of like Excel for Python)

Numpy: For numerical operation

glob: For file system operations

In [1]:
import pandas as pd
import numpy as np
from glob import glob 


## Read a Sample File - Earnings
Using pd.read_csv(), we can read a file into the system, and doing .head() on the imported data will show you a few rows. The parameter (10) that we pass to head() controls the number of rows that you want to see

In [2]:
sample_file = pd.read_csv("../Week 3/estimize/ADBE_estimize.csv")
sample_file.head(10)

,date,ticker,eps,wallstreet_estimates,fiscal_year,fiscal_quarter
0,2014-12-11,ADBE,0.36,0.299,2014,4
1,2014-09-16,ADBE,0.28,0.259,2014,3
2,2014-06-17,ADBE,0.37,0.294,2014,2
3,2014-03-18,ADBE,0.30,0.253,2014,1
4,2013-12-12,ADBE,0.32,0.318,2013,4
5,2013-09-17,ADBE,0.32,0.336,2013,3
6,2013-06-18,ADBE,0.36,0.332,2013,2
7,2013-03-19,ADBE,0.35,0.310,2013,1
8,2012-12-13,ADBE,0.61,0.570,2012,4
9,2012-09-19,ADBE,0.58,0.580,2012,3


As you can see, the data contains a date (the date that the earnings were released), the ticker code of the stock, the consensus wall street estimate for the EPS in that month, and the fiscal year and quarter corresponding to the estimate. 

We have this data for 39 Major Tech Companies (listed in finviz_tech.csv), for each quarter from 2000 - 2014. The data was sourced from https://data.estimize.com. The code that pulls together the data can be found in estimize.py

## Read a Sample File - Stock Price History
Agian, we use read_csv and head, to look at our stocks data

In [3]:
sample_file = pd.read_csv("../Week 3/stocks/ADBE.csv")
sample_file.head(10)

,date,ticker,open,high,low,close,volume
0,2000-01-03,ADBE,16.69,16.76,15.95,16.27,7384400
1,2000-01-04,ADBE,15.64,16.34,14.88,14.91,7813200
2,2000-01-05,ADBE,14.46,15.58,14.46,15.20,14927200
3,2000-01-06,ADBE,15.25,15.55,15.05,15.33,10221200
4,2000-01-07,ADBE,15.28,16.07,15.11,16.07,8253200
5,2000-01-10,ADBE,16.23,16.69,15.79,16.69,10987200
6,2000-01-11,ADBE,16.68,16.69,15.48,15.55,9616000
7,2000-01-12,ADBE,15.58,15.73,15.33,15.47,8051200
8,2000-01-13,ADBE,15.59,16.38,15.58,16.29,5527200
9,2000-01-14,ADBE,16.54,16.72,16.29,16.46,5136800


In this dataset, we can see that we have the daily Open, High, Low, Close, and Trading Volume associated with the stock. This data set was pulled for the same 39 companies and the same time period and was sourced from Yahoo Finance. The code that pulls together the data can be found in yahoo_quote.py

## Setting up the Problem

In order to create a predictive model that "learns" from historical data (and can subsequently used to forecast future events), we must first define a "Target" variable that can be fed in as the learning objective. To do this, let's first load each Earnings dataset, and combine them all into one large dataframe.

For this we will use the pd.concat (i.e. concatenate) function


In [9]:
estimize_files = glob('../Week 3/estimize/*')

In [10]:
all_data = []
for f in estimize_files:
    df = pd.read_csv(f, na_values = ["None"])
    #Convert the "date" column from string to date format
    df["date"] = pd.to_datetime(df["date"])
    all_data.append(df)
    
combined_dataset = pd.concat(all_data)

Now, let's look at whether we were successfull. By using value_counts() on a column, we can see how many ocurrences of each value are found. We can see here that for most of our companies we have all of the data (4 quarters per year for 15 years). But for some, we have a few missing years/quarters.

In [11]:
combined_dataset["ticker"].value_counts()

LRCX     60
KLAC     60
EQIX     60
CTSH     60
NVDA     60
CERN     60
FFIV     60
TXN      60
QCOM     60
SNDK     60
AKAM     60
ALTR     60
VRSN     60
CTXS     60
MSFT     60
ADBE     60
LLTC     60
SYMC     60
ADSK     60
MCHP     60
ADI      60
FLIR     60
INTU     60
MU       60
BRCM     60
ADP      60
WDC      60
NTAP     60
CA       60
CSCO     60
AMAT     60
EA       60
YHOO     60
INTC     60
XLNX     60
ATVI     58
SWKS     47
FTR      44
GOOGL    44
Name: ticker, dtype: int64

Now, let's define the objective or "target" variable. We will call this variable "beat" and define it as cases where the actual EPS was higher than the wall street consensus EPS. This will allow us to train a predictor that can, in the future, help us forecast if the EPS is going to be higher than the consensus estimate. 

Mathematically, what we are building is called two-class supervised learning classifier. The model takes inputs and predicts the probabilities that the output is one of two "classes" - in our case, whether the company will "beat" the EPS consensus or not. We need to label the classes in binary form: following convention, we will label the class that we are interested in as a 1, and the opposite case a 0 (i.e. when EPS > Consensus, then "beat" = 1, else 0). 

To do this, let's use np.where

In [12]:
combined_dataset["beat"] = np.where(combined_dataset["eps"] > combined_dataset["wallstreet_estimates"], 1, 0)

Now, let's take a look at our modified data frame

In [13]:
combined_dataset.head(10)

,date,ticker,eps,wallstreet_estimates,fiscal_year,fiscal_quarter,beat
0,2014-10-29,FFIV,1.57,1.477,2014,4,1
1,2014-07-23,FFIV,1.39,1.348,2014,3,1
2,2014-04-23,FFIV,1.27,1.250,2014,2,1
3,2014-01-22,FFIV,1.22,1.193,2014,1,1
4,2013-10-23,FFIV,1.26,1.188,2013,4,1
5,2013-07-24,FFIV,1.12,1.083,2013,3,1
6,2013-04-24,FFIV,1.07,1.088,2013,2,0
7,2013-01-23,FFIV,1.14,1.150,2013,1,0
8,2012-10-24,FFIV,1.12,1.180,2012,4,0
9,2012-07-18,FFIV,1.14,1.140,2012,3,0


And to establish a baseline, we can again use value_counts() to see the true underlying distribution of beat

In [14]:
combined_dataset["beat"].value_counts()

1    1528
0     765
Name: beat, dtype: int64

We can see that in about 66% of the cases, the actual EPS is better than the consensus. So, if we were to guess that each and every one of the earnings would beat the consensus estimate, we would be correct 66% of the time. Let's keep this fact in mind and build a model to see if we can get better than this

Finally, let's store the output

In [15]:
combined_dataset.to_csv("combined_dataset.csv")